In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../../../')
print(os.getcwd())
from load_config import load_config
from v2.strategy.indicators.optimal_v2 import Optimal_v2
from v2.model import Trading
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.roc import RateOfChange
from v2.strategy.indicators.notebook_utils import fetchIndicators, genDataForAll, generateSpans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier


/home/marshingjay/Repos/vivaldi/back_testing


In [2]:
# features = ["RSI", "MACD", "stosc_k", "stosc_d", "RateOfChange_MACD_1", "RateOfChange_MACD_5", "RateOfChange_MACD_10", "RateOfChange_MACD_60", "RateOfChange_RSI_1", "RateOfChange_RSI_5", "RateOfChange_RSI_10", "RateOfChange_RSI_60", "RateOfChange_close_1", "RateOfChange_close_5", "RateOfChange_close_10", "RateOfChange_close_60"]


In [3]:


def filter_optimal(optimal):
    if optimal > 0.75:
        return 1.0
    # elif optimal < -0.99:
    #     return -1.0
    else:
        return 0.0


model = Trading(load_config('config.hjson'))

dataset_list = []
for g,n in model.df_groups:
    print(n)
    for d in g:
        features = ["volume", "RSI", "slowk", "slowd"]
        cur_dataset = d
        my_inds = fetchIndicators(["rsi", "macd", "stochastic_oscillator", "optimal_v2"], param_specification={
            'macd.signal': 360,
            'macd.ema_slow': 480,
            'macd.ema_fast': 240,
            'ema.period': 360,
            'sma.period': 360,
            'stochastic_oscillator.highlow': 360,
            'stochastic_oscillator.k': 1080,
            'rsi.period': 360,
            'bollinger_bands.period': 360
        })
        genDataForAll(cur_dataset, my_inds)
        # cur_dataset.dropna(inplace=True)
        features.extend(generateSpans(cur_dataset, 'roc', 'RSI', 'period', [1,5,10,60]))
        features.extend(generateSpans(cur_dataset, 'roc', 'MACD', 'period', [1,5,10,60]))
        features.extend(generateSpans(cur_dataset, 'roc', 'close', 'period', [1,5,10,60]))
        
        # roc_5 = RateOfChange(_params=[Param(0,0,0,'period', 5)], _appended_name='RSI_5')
        # roc_1 = RateOfChange(_params=[Param(0,0,0,'period', 1)], _appended_name='RSI_1')
        # roc_10 = RateOfChange(_params=[Param(0,0,0,'period',10)], _appended_name='RSI_10')
        # roc_60 = RateOfChange(_params=[Param(0,0,0,'period', 60)], _appended_name='RSI_60')
        # roc_1.genData(cur_dataset, gen_new_values=False, value='RSI')
        # roc_5.genData(cur_dataset, gen_new_values=False, value='RSI')
        # roc_10.genData(cur_dataset, gen_new_values=False, value='RSI')
        # roc_60.genData(cur_dataset, gen_new_values=False, value='RSI')
        # roc_5 = RateOfChange(_params=[Param(0,0,0,'period', 5)], _appended_name='MACD_5')
        # roc_1 = RateOfChange(_params=[Param(0,0,0,'period', 1)], _appended_name='MACD_1')
        # roc_10 = RateOfChange(_params=[Param(0,0,0,'period',10)], _appended_name='MACD_10')
        # roc_60 = RateOfChange(_params=[Param(0,0,0,'period', 60)], _appended_name='MACD_60')
        # roc_1.genData(cur_dataset, gen_new_values=False, value='MACD')
        # roc_5.genData(cur_dataset, gen_new_values=False, value='MACD')
        # roc_10.genData(cur_dataset, gen_new_values=False, value='MACD')
        # roc_60.genData(cur_dataset, gen_new_values=False, value='MACD')
        # roc_5 = RateOfChange(_params=[Param(0,0,0,'period', 5)], _appended_name='close_5')
        # roc_1 = RateOfChange(_params=[Param(0,0,0,'period', 1)], _appended_name='close_1')
        # roc_10 = RateOfChange(_params=[Param(0,0,0,'period',10)], _appended_name='close_10')
        # roc_60 = RateOfChange(_params=[Param(0,0,0,'period', 60)], _appended_name='close_60')
        # roc_1.genData(cur_dataset, gen_new_values=False, value='close')
        # roc_5.genData(cur_dataset, gen_new_values=False, value='close')
        # roc_10.genData(cur_dataset, gen_new_values=False, value='close')
        # roc_60.genData(cur_dataset, gen_new_values=False, value='close')
        cur_dataset['optimal'] = cur_dataset.apply(lambda x: filter_optimal(x.optimal), axis=1)
        q_scalar = QuantileTransformer(n_quantiles=200)

        cur_dataset.dropna(inplace=True)
        cur_dataset[features] = q_scalar.fit_transform(cur_dataset[features])
        dataset_list.append(cur_dataset)
        break

        
    # create one minute, five minute, ten minute, and twenty minute slopes for macd, rsi




BTCUSDT-1m
roc 1 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 5 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 10 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 60 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 1 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 5 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 10 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 60 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 1 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 5 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 10 period <class 'v2.strategy.indicators.roc.RateOfChange'>
roc 60 period <class 'v2.strategy.indicators.roc.RateOfChange'>


In [4]:
cur_dataset

...                                 0.004777   
177053              ...                                 0.007254   
177054              ...                                 0.005501   
177055              ...                                 0.004590   
177056              ...                                 0.004678   
177057              ...                                 0.001804   
177058              ...                                -0.003289   
177059              ...                                -0.005384   
177060              ...                                -0.006787   
177061              ...                                 0.001872   
177062              ...                                 0.000299   
177063              ...                                -0.001013   

        RateOfChange_RSI_period_60  RateOfChange_MACD_period_1  \
1297                     -0.044688                    0.013202   
1298                     -0.044688                    0.012839   
1299                     -0.034516                    0.008787   
1300                     -0.033631                    0.008572   
1301                     -0.028498                    0.006511   
1302                     -0.025015                    0.006356   
1303                     -0.012703                    0.006206   
1304                     -0.025492                    0.006059   
1305                     -0.021644                    0.004568   
1306                     -0.019474                    0.004458   
1307                     -0.010867                    0.004348   
1308                      0.001483                    0.004243   
1309                     -0.011605                    0.004139   
1310                     -0.009663                    0.004038   
1311                     -0.000650                    0.000908   
1312                      0.017635                    0.000862   
1313                      0.010716                   -0.001656   
1314                      0.037904                   -0.001674   
1315                      0.011286                   -0.001130   
1316                     -0.011941                    0.002955   
1317                     -0.011941                    0.002877   
1318                      0.002239                    0.001293   
1319                      0.010509                    0.001241   
1320                      0.010509                    0.001190   
1321                      0.010509                    0.001139   
1322                      0.001608                    0.001090   
1323                      0.009730                    0.001040   
1324                      0.017604                    0.001711   
1325                      0.017865                    0.001653   
1326                      0.007382                    0.001596   
...                            ...                         ...   
177034                   -0.014234                   -0.023896   
177035                   -0.013040                   -0.020111   
177036                   -0.002921                   -0.011995   
177037                   -0.003626                   -0.013061   
177038                   -0.010207                   -0.020211   
177039                   -0.006859                   -0.013175   
177040                   -0.009732                   -0.016339   
177041                   -0.008538                   -0.013001   
177042                   -0.014392                   -0.016538   
177043                   -0.013056                   -0.018356   
177044                   -0.010968                   -0.012302   
177045                   -0.014345                   -0.016576   
177046                   -0.017296                   -0.012384   
177047                   -0.017111                   -0.012110   
177048                   -0.019909                   -0.011167   
177049                   -0.018223                   -0.007554   
177050                   -0.016933 

In [5]:
final_dataset = pd.concat(dataset_list)

In [6]:
dataset_list = []

In [7]:
final_dataset

...                                 0.004777   
177053              ...                                 0.007254   
177054              ...                                 0.005501   
177055              ...                                 0.004590   
177056              ...                                 0.004678   
177057              ...                                 0.001804   
177058              ...                                -0.003289   
177059              ...                                -0.005384   
177060              ...                                -0.006787   
177061              ...                                 0.001872   
177062              ...                                 0.000299   
177063              ...                                -0.001013   

        RateOfChange_RSI_period_60  RateOfChange_MACD_period_1  \
1297                     -0.044688                    0.013202   
1298                     -0.044688                    0.012839   
1299                     -0.034516                    0.008787   
1300                     -0.033631                    0.008572   
1301                     -0.028498                    0.006511   
1302                     -0.025015                    0.006356   
1303                     -0.012703                    0.006206   
1304                     -0.025492                    0.006059   
1305                     -0.021644                    0.004568   
1306                     -0.019474                    0.004458   
1307                     -0.010867                    0.004348   
1308                      0.001483                    0.004243   
1309                     -0.011605                    0.004139   
1310                     -0.009663                    0.004038   
1311                     -0.000650                    0.000908   
1312                      0.017635                    0.000862   
1313                      0.010716                   -0.001656   
1314                      0.037904                   -0.001674   
1315                      0.011286                   -0.001130   
1316                     -0.011941                    0.002955   
1317                     -0.011941                    0.002877   
1318                      0.002239                    0.001293   
1319                      0.010509                    0.001241   
1320                      0.010509                    0.001190   
1321                      0.010509                    0.001139   
1322                      0.001608                    0.001090   
1323                      0.009730                    0.001040   
1324                      0.017604                    0.001711   
1325                      0.017865                    0.001653   
1326                      0.007382                    0.001596   
...                            ...                         ...   
177034                   -0.014234                   -0.023896   
177035                   -0.013040                   -0.020111   
177036                   -0.002921                   -0.011995   
177037                   -0.003626                   -0.013061   
177038                   -0.010207                   -0.020211   
177039                   -0.006859                   -0.013175   
177040                   -0.009732                   -0.016339   
177041                   -0.008538                   -0.013001   
177042                   -0.014392                   -0.016538   
177043                   -0.013056                   -0.018356   
177044                   -0.010968                   -0.012302   
177045                   -0.014345                   -0.016576   
177046                   -0.017296                   -0.012384   
177047                   -0.017111                   -0.012110   
177048                   -0.019909                   -0.011167   
177049                   -0.018223                   -0.007554   
177050                   -0.016933 

In [8]:
# import joblib

# joblib.dump(mm_scalar, './mm_scalar.sav')

In [9]:


train, test = train_test_split(final_dataset, test_size=0.05, shuffle=False)


<h3>Models</h3>
<p>Random Forest</p>
<p>Fine tuned RF</p>
<p>Gradient boost</p>
<p>GB inside RF</p>
<p>RF inside GB</p>
<p>LSTM</p>

In [10]:

buy_signals = train[train['optimal'] == 1.0]

In [11]:
mean_num_singals = int(len(buy_signals))

In [12]:
not_signals = train[train['optimal'] == 0.0]


In [13]:
print(mean_num_singals, len(not_signals))

6028 160950


In [14]:
not_signals_sampled = not_signals.sample(n=mean_num_singals * 8, random_state=69420, axis=0)
not_signals_sampled = not_signals_sampled.sample(frac=1)

In [15]:

balanced_data_buy = pd.concat([not_signals_sampled, buy_signals])
not_signals_sampled = []
buy_signals = []
not_signals = []


In [16]:
# filtered_balanced_data_buy = balanced_data_buy[["close", "stosc_k", "stosc_d", "slope", 'rsi']]
filtered_balanced_data_buy = balanced_data_buy[features]




balanced_data_optimal_buy = balanced_data_buy[['optimal']]
balanced_data_buy = []



In [17]:
filtered_balanced_data_buy

volume       RSI     slowk     slowd
37841   0.633312  0.223797  0.154690  0.619384
84151   0.301899  0.504758  0.639268  0.699279
143610  0.589674  0.509043  0.281337  0.502454
32442   0.330303  0.289287  0.237108  0.135559
37692   0.363486  0.245420  0.357551  0.874015
103235  0.756103  0.242636  0.746806  0.306696
98046   0.672823  0.139343  0.367496  0.238145
162453  0.995835  0.417780  0.697026  0.867700
124844  0.805821  0.053257  0.179079  0.188430
111558  0.545460  0.628143  0.403948  0.424569
29050   0.623674  0.801288  0.418418  0.777144
68300   0.734003  0.536380  0.546710  0.227944
99557   0.000000  0.406117  0.424818  0.513468
32110   0.501652  0.034454  0.158013  0.270622
52247   0.000000  0.139053  0.036527  0.096546
86354   0.499228  0.413563  0.119567  0.581535
1770    0.000000  0.516660  0.613935  0.553465
64557   0.366422  0.384012  0.113994  0.245845
146115  0.517769  0.729023  0.739772  0.876140
3082    0.192045  0.032192  0.610323  0.319975
27997   0.407790  0.771182  0.948738  0.800735
103869  0.477446  0.243165  0.185500  0.189654
157060  0.393901  0.098579  0.451960  0.798401
158209  0.878560  0.874172  0.776943  0.909739
149726  0.873139  0.964174  0.890827  0.951995
58945   0.000000  0.437214  0.707978  0.388938
73819   0.000000  0.491692  0.184100  0.057579
162039  0.952434  0.965088  0.758475  0.915064
25984   0.206536  0.300113  0.082561  0.487479
56193   0.268295  0.629918  0.606811  0.662085
...          ...       ...       ...       ...
167289  0.959985  0.462927  0.493998  0.841591
167333  0.906984  0.679884  0.446312  0.807292
167377  0.995063  0.560454  0.449460  0.762335
167430  0.921580  0.491648  0.436723  0.703949
167527  0.859950  0.763986  0.523228  0.591114
167622  0.995692  0.970081  0.742996  0.554317
167645  0.959244  0.949066  0.756472  0.559816
167661  0.875849  0.856472  0.766757  0.564860
167682  0.965102  0.928609  0.775614  0.572168
167714  0.960500  0.877141  0.825014  0.589382
167735  0.946523  0.781981  0.834378  0.605870
167746  0.933106  0.755574  0.831739  0.614577
167760  0.967983  0.726052  0.839051  0.625562
167793  0.952837  0.360862  0.846989  0.653121
167812  0.967803  0.242556  0.834238  0.670215
167841  0.761761  0.202883  0.771138  0.692735
167885  0.729799  0.321420  0.669416  0.717743
167900  0.775446  0.353854  0.630589  0.723052
167970  0.963162  0.373773  0.571968  0.744024
168011  0.868936  0.102602  0.438624  0.746262
168031  0.986798  0.076308  0.363337  0.742533
168058  0.916759  0.100706  0.267610  0.729605
168079  0.727791  0.139893  0.214597  0.713976
168098  0.875691  0.089070  0.174923  0.691934
168128  0.989608  0.106922  0.111755  0.647832
168150  0.929949  0.123739  0.083162  0.604794
168162  0.820080  0.163719  0.072581  0.578697
168196  0.767603  0.229461  0.051482  0.494339
168217  0.857398  0.283791  0.039954  0.441481
168265  0.995346  0.345927  0.032381  0.326448

[54252 rows x 4 columns]

In [18]:

clf_buy = RandomForestClassifier(random_state=69420, n_jobs=-1, n_estimators=100, criterion='gini', min_samples_leaf=2)

clf_buy.fit(filtered_balanced_data_buy, balanced_data_optimal_buy)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=69420,
                       verbose=0, warm_start=False)

In [19]:
def insert_buys(close, prediction):
    if prediction == 1:
        return close
    else:
        return None

def insert_sells(close, prediction):
    if prediction == -1:
        return close
    else:
        return None



In [20]:
import pickle

In [21]:
filtered_test = test[["close", 'optimal', 'volume']]
filtered_test_2 = test[features]

In [22]:
# filtered_sell = test[["close", "rsi", "MACD", "stosc_k", "stosc_d", "slope", "variance"]]

In [23]:
# sell_model = pickle.load(open('./v2/strategy/saved_models/optimal_v2_sell_rf.sav', 'rb'))

In [24]:
filtered_test['predict_buy'] = clf_buy.predict(filtered_test_2.values)
# filtered_test['predict_sell'] = sell_model.predict(filtered_sell)

In [25]:
len(filtered_test)

8789

In [26]:
len(filtered_test[filtered_test['predict_buy'] != 0.0])

37

In [27]:

filtered_test['buy'] = filtered_test.apply(lambda x: insert_buys(x.close, x.predict_buy), axis=1)
filtered_test['optimal_buy'] = filtered_test.apply(lambda x: insert_buys(x.close, x.optimal), axis=1)
# filtered_test['sell'] = filtered_test.apply(lambda x: insert_sells(x.close, x.predict_sell), axis=1)



In [28]:
first_amnt

NameError: name 'first_amnt' is not defined

In [80]:
from v2.strategy.indicators.bollinger_bands import BollingerBands

In [81]:
bollinger_bands_params = Param(0,0,0,'period',175)
boll = BollingerBands(_params=[bollinger_bands_params])
boll.genData(filtered_test, gen_new_values=False)


In [82]:
reset_filtered = filtered_test.reset_index(drop=True)

In [83]:
# reset_filtered

In [84]:
# reset_filtered['count'] = reset_filtered['sell'].rolling(60).count() - reset_filtered['buy'].rolling(60).count() 

In [85]:
# reset_filtered

In [99]:
first_amnt = reset_filtered[56000:60000]
first_amnt['volume'] = first_amnt['volume'].rolling(window=10).mean()

In [100]:
import matplotlib.pyplot as plt
plt.clf()
fig, ax1 = plt.subplots()

plt.figure(figsize=(20,10))
ax1.scatter(x=first_amnt.index, y=first_amnt['buy'], color='red')
ax1.plot(first_amnt.index, first_amnt['boll_lower'], color='green')
ax1.plot(first_amnt.index, first_amnt['boll_upper'], color='purple')
ax2 = ax1.twinx()
ax2.plot(first_amnt.index, first_amnt['volume'], color='black')

# plt.scatter(x=first_amnt.index, y=first_amnt['optimal_buy'], color='purple')

# plt.scatter(x=first_amnt.index, y=first_amnt['sell'], color='green')

ax1.plot(first_amnt.index, first_amnt['close'], color='blue')
plt.show()

<Figure size 432x288 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

In [101]:
# plt.figure(figsize=(20,10))
# plt.plot(first_amnt.index, first_amnt['count'].rolling(480).mean(), color='purple')
# plt.plot(first_amnt.index, first_amnt['count'].rolling(1440).mean(), color='blue')
# plt.plot(first_amnt.index, first_amnt['count'].rolling(4320).mean(), color='orange')
# plt.show()